In [2]:
import enum
import warnings
from typing import Optional, Tuple

import torch
import torch.nn.functional as F

from kornia.constants import pi
from kornia.core import Tensor, concatenate, pad, stack, tensor, where
from kornia.core.check import KORNIA_CHECK, KORNIA_CHECK_SHAPE
from kornia.utils.helpers import _torch_inverse_cast
from kornia.utils.misc import eye_like



def denormalize_pixel_coordinates3d(
        pixel_coordinates: torch.Tensor,
        depth: int,
        height: int,
        width: int,
        eps: float = 1e-8) -> torch.Tensor:
    r"""Denormalize pixel coordinates.
    The input is assumed to be -1 if on extreme left, 1 if on
    extreme right (x = w-1).
    Args:
        pixel_coordinates (torch.Tensor): the normalized grid coordinates.
          Shape can be :math:`(*, 3)`.
        depth (int): the maximum depth in the x-axis.
        height (int): the maximum height in the y-axis.
        width (int): the maximum width in the x-axis.
        eps (float): safe division by zero. (default 1e-8).
    Return:
        torch.Tensor: the denormalized pixel coordinates.
    """
    if pixel_coordinates.shape[-1] != 3:
        raise ValueError("Input pixel_coordinates must be of shape (*, 3). "
                         "Got {}".format(pixel_coordinates.shape))
    # compute normalization factor
    dhw: torch.Tensor = torch.stack([
        torch.tensor(depth), torch.tensor(width), torch.tensor(height)
    ]).to(pixel_coordinates.device).to(pixel_coordinates.dtype)

    factor: torch.Tensor = torch.tensor(2.) / (dhw - 1).clamp(eps)

    return torch.tensor(1.) / factor * (pixel_coordinates + 1)